# Query Gaia DR3 Catalog and Convert to Synthetic Photometries

In [1]:
import numpy as np

## Function

In [2]:
def calculate_snr(flux, flux_error):
    snr = flux / flux_error
    return snr

def calculate_magnitude_error(flux, flux_error):
    snr = calculate_snr(flux, flux_error)
    magnitude_error = 2.5*np.log10(1+(1/snr))
    
    return magnitude_error


In [144]:
def query_gaiaxp_continuous_raw(str_to_query, radeg, decdeg, query_radius, fname):
	
	example_query = f"""SELECT {str_to_query}, DISTANCE({radeg},{decdeg}, ra, dec) AS ang_sep
	FROM gaiadr3.gaia_source
	WHERE DISTANCE({radeg},{decdeg}, ra, dec) < {query_radius}
	AND has_epoch_photometry = 'True'
	AND has_xp_continuous = 'True'
	AND non_single_star = 0"""

	#	Most Sources are variable... 
	# AND phot_variable_flag = 'CONSTANT'"""

	job = gaia.launch_job_async(example_query, dump_to_file=False)
	ids = job.get_results()

	# Now retrieve the BP/RP mean spectra in the continuous representation
	result = gaia.load_data(
		ids=ids['source_id'], 
		format='csv', 
		data_release='Gaia DR3', 
		data_structure='raw', 
		retrieval_type='XP_CONTINUOUS', 
		avoid_datatype_check=True
		)

	# Result will be a dictionary, so you can check the available keys by running result.keys() # In this example we are looking in particular for the XP_CONTINUOUS_RAW key
	continuous_key = [key for key in result.keys() if 'continuous' in key.lower()][0]

	# The first element is the result we want as an Astropy table
	data = result[continuous_key][0]

	# Astropy has a 'write' method for tables
	# Write the table to CSV
	data.write(fname, format='csv', overwrite=True)

	return ids.to_pandas()

## Ready

In [3]:
from gaiaxpy import generate, PhotometricSystem
from gaiaxpy import PhotometricSystem, load_additional_systems
path_to_filterset = '../config/7DT'
PhotometricSystem = load_additional_systems(path_to_filterset)
PhotometricSystem.get_available_systems().split(', ')[-2:]

Loading systems... Additional systems version is: v1.
Systems loaded. Use PhotometricSystem.get_available_systems() to get the names of the current available systems.


['USER_7DT_Cal', 'USER_7DT_Edmund']

In [4]:
PhotometricSystem.get_available_systems()

'DECam, Els_Custom_W09_S2, Euclid_VIS, Gaia_2, Gaia_DR3_Vega, Halpha_Custom_AB, H_Custom, Hipparcos_Tycho, HST_ACSWFC, HST_HUGS_Std, HST_WFC3UVIS, HST_WFPC2, IPHAS, JKC, JKC_Std, JPAS, JPLUS, JWST_NIRCAM, LSST, PanSTARRS1, PanSTARRS1_Std, Pristine, SDSS, SDSS_Std, Sky_Mapper, Stromgren, Stromgren_Std, WFIRST, USER_7DT_Cal, USER_7DT_Edmund'

In [5]:
phot_system = PhotometricSystem.USER_7DT_Edmund

## Query Gaia DR3

- Login to the Gaia server

In [ ]:
user = 'gpaek'
password = 'Silenthill0!'

In [6]:
from astroquery.gaia import GaiaClass

# Connect to Gaia archive
gaia = GaiaClass(
	gaia_tap_server='https://gea.esac.esa.int/',
	gaia_data_server='https://gea.esac.esa.int/',
	)
gaia.login(
	user=user,
    password=password'
)

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


- Query Configuration

In [140]:
tname = 'test'
radeg = 266.41683
decdeg = -29.00781
query_radius = 5/60.

fname = f"XP_CONTINUOUS_RAW_{tname}.csv"

In [8]:
columns_to_query = [
	'gaia_source.solution_id',
	'gaia_source.source_id',
	'gaia_source.ra',
	'gaia_source.dec',
	'gaia_source.parallax',
	'gaia_source.l',
	'gaia_source.b',

	'gaia_source.phot_g_mean_mag',
	'gaia_source.bp_rp',
	'gaia_source.bp_g',
	'gaia_source.g_rp',

	'gaia_source.phot_variable_flag',
	# 'gaia_source.in_galaxy_candidates',
	'gaia_source.non_single_star',
	'gaia_source.has_xp_continuous',
	# 'gaia_source.has_xp_sampled',
	'gaia_source.has_rvs',
	'gaia_source.has_epoch_photometry',
	'gaia_source.ebpminrp_gspphot'
	]

str_to_query = ', '.join(columns_to_query)
print(f"{len(columns_to_query)} columns to query:")
print(str_to_query)
# str_to_query = '*'

17 columns to query:
gaia_source.solution_id, gaia_source.source_id, gaia_source.ra, gaia_source.dec, gaia_source.parallax, gaia_source.l, gaia_source.b, gaia_source.phot_g_mean_mag, gaia_source.bp_rp, gaia_source.bp_g, gaia_source.g_rp, gaia_source.phot_variable_flag, gaia_source.non_single_star, gaia_source.has_xp_continuous, gaia_source.has_rvs, gaia_source.has_epoch_photometry, gaia_source.ebpminrp_gspphot


- Query

In [145]:
metadf = query_gaiaxp_continuous_raw(str_to_query, radeg, decdeg, query_radius, fname)

INFO: Query finished. [astroquery.utils.tap.core]


In [9]:


# Run your ADQL query to get a list of source_ids
# example_query = "select TOP 10 source_id from gaiadr3.gaia_source where has_xp_continuous = 'True'"
# example_query = """SELECT TOP 10 gaia_source.solution_id,gaia_source.source_id,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.phot_g_mean_mag,gaia_source.bp_rp,gaia_source.bp_g,gaia_source.g_rp,gaia_source.phot_variable_flag,gaia_source.l,gaia_source.b,gaia_source.in_galaxy_candidates,gaia_source.non_single_star,gaia_source.has_xp_continuous,gaia_source.has_xp_sampled,gaia_source.has_rvs,gaia_source.has_epoch_photometry,gaia_source.ebpminrp_gspphot
# FROM gaiadr3.gaia_source 
# WHERE has_xp_continuous = 'True'
# CONTAINS(
# 	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),
# 	CIRCLE(
# 		'ICRS',
# 		COORD1(EPOCH_PROP_POS(28.70945968,-27.47659421,7.3358,310.9670,-232.0660,101.1400,2000,2016.0)),
# 		COORD2(EPOCH_PROP_POS(28.70945968,-27.47659421,7.3358,310.9670,-232.0660,101.1400,2000,2016.0)),
# 		1.0)
# )=1"""

example_query = f"""SELECT {str_to_query}, DISTANCE({radeg},{decdeg}, ra, dec) AS ang_sep
FROM gaiadr3.gaia_source
WHERE DISTANCE({radeg},{decdeg}, ra, dec) < {query_radius}
AND has_epoch_photometry = 'True'
AND has_xp_continuous = 'True'
AND non_single_star = 0"""

#	Most Sources are variable... 
# AND phot_variable_flag = 'CONSTANT'"""

job = gaia.launch_job_async(example_query,dump_to_file=False)
ids = job.get_results()

# Now retrieve the BP/RP mean spectra in the continuous representation
result = gaia.load_data(
	ids=ids['source_id'], 
	format='csv', 
	data_release='Gaia DR3', 
	data_structure='raw', 
	retrieval_type='XP_CONTINUOUS', 
	avoid_datatype_check=True
	)

# Result will be a dictionary, so you can check the available keys by running result.keys() # In this example we are looking in particular for the XP_CONTINUOUS_RAW key
continuous_key = [key for key in result.keys() if 'continuous' in key.lower()][0]

# The first element is the result we want as an Astropy table
data = result[continuous_key][0]

# Astropy has a 'write' method for tables
# Write the table to CSV
data.write(fname, format='csv', overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


In [10]:
metatbl = ids
metadf = metatbl.to_pandas()

- Lsst_mag_g
Lsst_mag_r
Lsst_mag_i
Lsst_mag_z
Lsst_mag_y

- Panstarrs1_mag_g
Panstarrs1_mag_rp
Panstarrs1_mag_ip
Panstarrs1_mag_zp
Panstarrs1_mag_yp

- Panstarrs1Std_mag_gp
Panstarrs1Std_mag_rp
Panstarrs1Std_mag_ip
Panstarrs1Std_mag_zp
Panstarrs1Std_mag_yp

- Sdss_mag_u
Sdss_mag_g
Sdss_mag_r
Sdss_mag_i
Sdss_mag_z

- SdssStd_mag_u
SdssStd_mag_g
SdssStd_mag_r
SdssStd_mag_i
SdssStd_mag_z

- SkyMapper_mag_u
SkyMapper_mag_u2
SkyMapper_mag_v
SkyMapper_mag_g
SkyMapper_mag_r
SkyMapper_mag_i
SkyMapper_mag_z

In [11]:
# PhotometricSystem.get_available_systems()

In [12]:
# _synphot = generate(fname, photometric_system=PhotometricSystem.Pristine)
# print()
# for key in _synphot.keys():
# 	if 'mag' in key:
# 		print(key)

In [48]:
phot_system_sdss = PhotometricSystem.SDSS
synthetic_photometry_sdss = generate(fname, photometric_system=phot_system_sdss)
synthetic_photometry_sdss

                                                               /11 [00:00<?, ?spec/s]

,source_id,Sdss_mag_u,Sdss_mag_g,Sdss_mag_r,Sdss_mag_i,Sdss_mag_z,Sdss_flux_u,Sdss_flux_g,Sdss_flux_r,Sdss_flux_i,Sdss_flux_z,Sdss_flux_error_u,Sdss_flux_error_g,Sdss_flux_error_r,Sdss_flux_error_i,Sdss_flux_error_z
0,4057468248915845504,17.728056,16.362976,15.512651,15.173541,14.960482,2.942914e-30,1.034689e-29,2.264329e-29,3.094451e-29,3.765367e-29,1.263716e-31,2.655134e-32,4.924079e-32,4.582103e-32,9.007572e-32
1,4057469627607446016,19.315338,17.748457,16.551108,15.958611,15.553781,6.821260e-31,2.888132e-30,8.700754e-30,1.501604e-29,2.180155e-29,1.314821e-31,2.184419e-32,5.609751e-32,7.474844e-32,1.604669e-31
2,4057470314802233728,20.462034,18.006844,16.812264,16.294394,15.931884,2.372392e-31,2.276473e-30,6.840605e-30,1.102154e-29,1.539028e-29,9.768099e-32,1.468128e-32,2.795960e-32,2.797579e-32,6.182535e-32
3,4057470344859979776,20.805606,18.602630,17.195209,16.535743,16.092675,1.728851e-31,1.315067e-30,4.807470e-30,8.824757e-30,1.327180e-29,1.046708e-31,1.430846e-32,3.276048e-32,2.768254e-32,7.152037e-32
4,4057470452241184128,19.642082,17.723193,16.549273,16.116660,15.858063,5.048554e-31,2.956126e-30,8.715472e-30,1.298184e-29,1.647308e-29,1.129101e-31,1.868633e-32,3.610798e-32,3.357000e-32,8.677165e-32
5,4057481580494165888,14.650161,13.180196,12.281513,11.869271,11.479202,5.011129e-29,1.940535e-28,4.440120e-28,6.490703e-28,9.296493e-28,7.541267e-31,2.010947e-31,4.600193e-31,4.553926e-31,8.284745e-31
6,4057482061536827008,13.339869,12.395776,12.017236,11.897319,11.801413,1.675145e-28,3.996592e-28,5.663772e-28,6.325172e-28,6.909311e-28,1.710705e-30,4.676613e-31,8.288265e-31,6.260668e-31,1.088370e-30
7,4057482306350939136,19.958589,18.273475,16.818868,16.271950,15.886128,3.771935e-31,1.780778e-30,6.799123e-30,1.125175e-29,1.605273e-29,9.568131e-32,1.421684e-32,2.793332e-32,2.697188e-32,5.997441e-32
8,4057482890466523136,18.886660,16.983006,15.914998,15.544319,15.301834,1.012363e-30,5.845178e-30,1.563150e-29,2.199240e-29,2.749581e-29,1.259385e-31,2.142476e-32,4.050196e-32,3.752722e-32,7.492252e-32
9,4057482989243742464,20.206612,17.470709,16.249175,15.887743,15.523776,3.001616e-31,3.730064e-30,1.149026e-29,1.602886e-29,2.241247e-29,1.546557e-31,3.163825e-32,9.802512e-32,1.439507e-31,3.185036e-31


In [49]:
phot_system_7dt = PhotometricSystem.USER_7DT_Edmund
synthetic_photometry_7dt = generate(fname, photometric_system=phot_system_7dt)
synthetic_photometry_7dt

                                                               /11 [00:00<?, ?spec/s]

,source_id,USER_7DT_Edmund_mag_375_50,USER_7DT_Edmund_mag_400_25,USER_7DT_Edmund_mag_412,USER_7DT_Edmund_mag_425_50,USER_7DT_Edmund_mag_425_25,USER_7DT_Edmund_mag_437,USER_7DT_Edmund_mag_450_25,USER_7DT_Edmund_mag_462,USER_7DT_Edmund_mag_475_25,...,USER_7DT_Edmund_flux_error_775_25,USER_7DT_Edmund_flux_error_787,USER_7DT_Edmund_flux_error_800_25,USER_7DT_Edmund_flux_error_812,USER_7DT_Edmund_flux_error_825_25,USER_7DT_Edmund_flux_error_837,USER_7DT_Edmund_flux_error_850_25,USER_7DT_Edmund_flux_error_862,USER_7DT_Edmund_flux_error_875_25,USER_7DT_Edmund_flux_error_887
0,4057468248915845504,17.668840,17.222670,16.927870,16.823825,16.869410,16.715995,16.466727,16.339192,16.276964,...,2.127312e-31,2.259905e-31,2.235918e-31,2.224255e-31,2.299377e-31,2.367483e-31,2.470824e-31,2.680260e-31,2.987561e-31,3.271469e-31
1,4057469627607446016,19.413507,18.882473,18.530390,18.361392,18.370671,18.194629,17.989741,17.781956,17.685444,...,3.376317e-31,3.566777e-31,3.617974e-31,3.623395e-31,3.704834e-31,3.964093e-31,4.153437e-31,4.631704e-31,5.227484e-31,5.799519e-31
2,4057470314802233728,19.745598,19.098286,18.805708,18.625117,18.747626,18.453251,18.196440,18.074033,17.923231,...,1.236767e-31,1.322610e-31,1.346780e-31,1.360872e-31,1.418867e-31,1.530538e-31,1.602413e-31,1.785599e-31,2.018845e-31,2.214899e-31
3,4057470344859979776,21.343611,20.190795,19.674083,19.403663,19.357892,19.176829,18.832480,18.630235,18.563962,...,1.184845e-31,1.236097e-31,1.250812e-31,1.259649e-31,1.303319e-31,1.373612e-31,1.468872e-31,1.670556e-31,1.943602e-31,2.232524e-31
4,4057470452241184128,19.512013,19.120260,18.678001,18.402402,18.508137,18.162456,17.771706,17.559673,17.486924,...,1.476730e-31,1.554168e-31,1.532942e-31,1.520865e-31,1.574672e-31,1.667818e-31,1.793780e-31,2.038844e-31,2.358841e-31,2.676879e-31
5,4057481580494165888,14.301338,13.768425,13.645308,13.565506,13.560671,13.483377,13.351183,13.224293,13.153600,...,1.932424e-30,2.057461e-30,2.099596e-30,2.118042e-30,2.235338e-30,2.384493e-30,2.565423e-30,2.873488e-30,3.203930e-30,3.499790e-30
6,4057482061536827008,13.051037,12.657594,12.585837,12.555891,12.559645,12.522986,12.429052,12.366806,12.371261,...,2.827665e-30,2.825089e-30,2.704769e-30,2.683429e-30,2.804493e-30,3.043432e-30,3.305120e-30,3.752811e-30,4.294264e-30,4.744009e-30
7,4057482306350939136,20.404213,19.822908,19.599520,19.130001,19.160955,18.766704,18.379910,18.111438,18.004647,...,1.182522e-31,1.249997e-31,1.264658e-31,1.272172e-31,1.307476e-31,1.362036e-31,1.434606e-31,1.582214e-31,1.775514e-31,1.939341e-31
8,4057482890466523136,18.753657,18.108740,17.800022,17.603166,17.702601,17.415927,17.056508,16.880474,16.776910,...,1.638968e-31,1.732117e-31,1.758090e-31,1.752015e-31,1.793246e-31,1.868642e-31,1.955131e-31,2.154513e-31,2.419309e-31,2.634995e-31
9,4057482989243742464,19.701995,18.820835,18.522804,18.303338,18.430368,18.107438,17.807507,17.519074,17.288420,...,6.450289e-31,6.718538e-31,6.816166e-31,6.989866e-31,7.269470e-31,7.502105e-31,8.005589e-31,8.675420e-31,9.682902e-31,1.074877e-30


## Merge Table

In [125]:
import pandas as pd
import numpy as np

In [126]:
synthetic_photometry = pd.merge(synthetic_photometry_sdss, synthetic_photometry_7dt, on='source_id')

In [127]:
merged_df = pd.merge(synthetic_photometry, metadf, on='source_id')
for key in merged_df.keys():
	print(key)

source_id
Sdss_mag_u
Sdss_mag_g
Sdss_mag_r
Sdss_mag_i
Sdss_mag_z
Sdss_flux_u
Sdss_flux_g
Sdss_flux_r
Sdss_flux_i
Sdss_flux_z
Sdss_flux_error_u
Sdss_flux_error_g
Sdss_flux_error_r
Sdss_flux_error_i
Sdss_flux_error_z
USER_7DT_Edmund_mag_375_50
USER_7DT_Edmund_mag_400_25
USER_7DT_Edmund_mag_412
USER_7DT_Edmund_mag_425_50
USER_7DT_Edmund_mag_425_25
USER_7DT_Edmund_mag_437
USER_7DT_Edmund_mag_450_25
USER_7DT_Edmund_mag_462
USER_7DT_Edmund_mag_475_25
USER_7DT_Edmund_mag_487
USER_7DT_Edmund_mag_500_25
USER_7DT_Edmund_mag_512
USER_7DT_Edmund_mag_525_25
USER_7DT_Edmund_mag_537
USER_7DT_Edmund_mag_550_25
USER_7DT_Edmund_mag_562
USER_7DT_Edmund_mag_575_25
USER_7DT_Edmund_mag_587
USER_7DT_Edmund_mag_600_25
USER_7DT_Edmund_mag_612
USER_7DT_Edmund_mag_625_25
USER_7DT_Edmund_mag_637
USER_7DT_Edmund_mag_650_25
USER_7DT_Edmund_mag_662
USER_7DT_Edmund_mag_675_25
USER_7DT_Edmund_mag_687
USER_7DT_Edmund_mag_700_25
USER_7DT_Edmund_mag_712
USER_7DT_Edmund_mag_725_25
USER_7DT_Edmund_mag_737
USER_7DT_Edmund_m

- Define Filtersets

In [128]:
filterlist_sdss = ['u', 'g', 'r', 'i', 'z',]
filterlist_7dt = []

for key in merged_df.keys():
	if 'USER_7DT_Edmund_mag_' in key:
		filte = key.replace('USER_7DT_Edmund_mag_', '')
		filterlist_7dt.append(filte)

In [129]:
#	SDSS
prefix_mag_sdss = 'Sdss_mag_'
prefix_flux_sdss = 'Sdss_flux_'
prefix_fluxerr_sdss = 'Sdss_flux_error_'
#	7DT
prefix_mag_7dt = 'USER_7DT_Edmund_mag_'
prefix_flux_7dt = 'USER_7DT_Edmund_flux_'
prefix_fluxerr_7dt = 'USER_7DT_Edmund_flux_error_'

In [130]:
key_to_rename_dict = {}

In [131]:
for filte in filterlist_sdss:
	#	New Values
	_mag = merged_df[f"{prefix_mag_sdss}{filte}"]
	_flux = merged_df[f"{prefix_flux_sdss}{filte}"]
	_fluxerr = merged_df[f"{prefix_fluxerr_sdss}{filte}"]
	_snr = calculate_snr(_flux, _fluxerr)
	_magerr = calculate_magnitude_error(_flux, _fluxerr)

	#	Former Keywords
	magkey = f"{prefix_mag_sdss}{filte}"
	fluxkey = f"{prefix_flux_sdss}{filte}"
	fluxerrkey = f"{prefix_fluxerr_sdss}{filte}"

	#	New Keywords
	newmagkey = f"{filte}_mag"
	newmagerrkey = f"{filte}_magerr"
	newfluxkey = f"{filte}_flux"
	newfluxerrkey = f"{filte}_fluxerr"
	newsnrkey = f"{filte}_snr"

	#	Keywords to Rename
	key_to_rename_dict[magkey] = newmagkey
	key_to_rename_dict[fluxkey] = newfluxkey
	key_to_rename_dict[fluxerrkey] = newfluxerrkey

	merged_df[newmagerrkey] = _magerr
	merged_df[newsnrkey] = _snr


In [132]:
for filte in filterlist_7dt:
	#	New Values
	_mag = merged_df[f"{prefix_mag_7dt}{filte}"]
	_flux = merged_df[f"{prefix_flux_7dt}{filte}"]
	_fluxerr = merged_df[f"{prefix_fluxerr_7dt}{filte}"]
	_snr = calculate_snr(_flux, _fluxerr)
	_magerr = calculate_magnitude_error(_flux, _fluxerr)

	#	Former Keywords
	magkey = f"{prefix_mag_7dt}{filte}"
	fluxkey = f"{prefix_flux_7dt}{filte}"
	fluxerrkey = f"{prefix_fluxerr_7dt}{filte}"

	#	New Keywords
	if '_50' in filte:
		replace_key = '_50'
		suffix = 'w'
	elif '_25' in filte:
		replace_key = '_25'
		suffix = ''
	else:
		replace_key = ''
		suffix = ''

	newmagkey = f"m{filte.replace(replace_key, suffix)}_mag"
	newmagerrkey = f"m{filte.replace(replace_key, suffix)}_magerr"
	newfluxkey = f"m{filte.replace(replace_key, suffix)}_flux"
	newfluxerrkey = f"m{filte.replace(replace_key, suffix)}_fluxerr"
	newsnrkey = f"m{filte.replace(replace_key, suffix)}_snr"

	#	Keywords to Rename
	key_to_rename_dict[magkey] = newmagkey
	key_to_rename_dict[fluxkey] = newfluxkey
	key_to_rename_dict[fluxerrkey] = newfluxerrkey

	merged_df[newmagerrkey] = _magerr
	merged_df[newsnrkey] = _snr

/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_21763/3730031010.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[newsnrkey] = _snr
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_21763/3730031010.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[newmagerrkey] = _magerr
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_21763/3730031010.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [133]:
key_to_rename_dict

{'Sdss_mag_u': 'u_mag',
 'Sdss_flux_u': 'u_flux',
 'Sdss_flux_error_u': 'u_fluxerr',
 'Sdss_mag_g': 'g_mag',
 'Sdss_flux_g': 'g_flux',
 'Sdss_flux_error_g': 'g_fluxerr',
 'Sdss_mag_r': 'r_mag',
 'Sdss_flux_r': 'r_flux',
 'Sdss_flux_error_r': 'r_fluxerr',
 'Sdss_mag_i': 'i_mag',
 'Sdss_flux_i': 'i_flux',
 'Sdss_flux_error_i': 'i_fluxerr',
 'Sdss_mag_z': 'z_mag',
 'Sdss_flux_z': 'z_flux',
 'Sdss_flux_error_z': 'z_fluxerr',
 'USER_7DT_Edmund_mag_375_50': 'm375w_mag',
 'USER_7DT_Edmund_flux_375_50': 'm375w_flux',
 'USER_7DT_Edmund_flux_error_375_50': 'm375w_fluxerr',
 'USER_7DT_Edmund_mag_400_25': 'm400_mag',
 'USER_7DT_Edmund_flux_400_25': 'm400_flux',
 'USER_7DT_Edmund_flux_error_400_25': 'm400_fluxerr',
 'USER_7DT_Edmund_mag_412': 'm412_mag',
 'USER_7DT_Edmund_flux_412': 'm412_flux',
 'USER_7DT_Edmund_flux_error_412': 'm412_fluxerr',
 'USER_7DT_Edmund_mag_425_50': 'm425w_mag',
 'USER_7DT_Edmund_flux_425_50': 'm425w_flux',
 'USER_7DT_Edmund_flux_error_425_50': 'm425w_fluxerr',
 'USER_7DT

In [134]:
merged_df.rename(columns=key_to_rename_dict, inplace=True)
for key in merged_df.keys():
	print(key)

source_id
u_mag
g_mag
r_mag
i_mag
z_mag
u_flux
g_flux
r_flux
i_flux
z_flux
u_fluxerr
g_fluxerr
r_fluxerr
i_fluxerr
z_fluxerr
m375w_mag
m400_mag
m412_mag
m425w_mag
m425_mag
m437_mag
m450_mag
m462_mag
m475_mag
m487_mag
m500_mag
m512_mag
m525_mag
m537_mag
m550_mag
m562_mag
m575_mag
m587_mag
m600_mag
m612_mag
m625_mag
m637_mag
m650_mag
m662_mag
m675_mag
m687_mag
m700_mag
m712_mag
m725_mag
m737_mag
m750_mag
m762_mag
m775_mag
m787_mag
m800_mag
m812_mag
m825_mag
m837_mag
m850_mag
m862_mag
m875_mag
m887_mag
m375w_flux
m400_flux
m412_flux
m425w_flux
m425_flux
m437_flux
m450_flux
m462_flux
m475_flux
m487_flux
m500_flux
m512_flux
m525_flux
m537_flux
m550_flux
m562_flux
m575_flux
m587_flux
m600_flux
m612_flux
m625_flux
m637_flux
m650_flux
m662_flux
m675_flux
m687_flux
m700_flux
m712_flux
m725_flux
m737_flux
m750_flux
m762_flux
m775_flux
m787_flux
m800_flux
m812_flux
m825_flux
m837_flux
m850_flux
m862_flux
m875_flux
m887_flux
m375w_fluxerr
m400_fluxerr
m412_fluxerr
m425w_fluxerr
m425_fluxerr
m437_f

In [135]:
# merged_df.to_csv(f'../output/gaia-{tname}.cat')

In [ ]:
def convert_gaiaxp_to_7DT_synphot(fname, metadf):
	#============================================================
	#	Synthetic Photometry
	#------------------------------------------------------------
	#	SDSS
	#------------------------------------------------------------
	phot_system_sdss = PhotometricSystem.SDSS
	synthetic_photometry_sdss = generate(fname, photometric_system=phot_system_sdss)
	synthetic_photometry_sdss
	#------------------------------------------------------------
	#	7DT
	#------------------------------------------------------------
	phot_system_7dt = PhotometricSystem.USER_7DT_Edmund
	synthetic_photometry_7dt = generate(fname, photometric_system=phot_system_7dt)
	synthetic_photometry_7dt
	#============================================================
	#	Merge the Table
	#------------------------------------------------------------
	#	Meta Table + SDSS + 7DT
	#------------------------------------------------------------
	synthetic_photometry = pd.merge(synthetic_photometry_sdss, synthetic_photometry_7dt, on='source_id')
	merged_df = pd.merge(synthetic_photometry, metadf, on='source_id')
	#------------------------------------------------------------
	#	Filter List
	#------------------------------------------------------------
	filterlist_sdss = ['u', 'g', 'r', 'i', 'z',]
	filterlist_7dt = []
	for key in merged_df.keys():
		if 'USER_7DT_Edmund_mag_' in key:
			filte = key.replace('USER_7DT_Edmund_mag_', '')
			filterlist_7dt.append(filte)
	#------------------------------------------------------------
	#	Rename configurations
	#------------------------------------------------------------
	#	SDSS
	prefix_mag_sdss = 'Sdss_mag_'
	prefix_flux_sdss = 'Sdss_flux_'
	prefix_fluxerr_sdss = 'Sdss_flux_error_'
	#	7DT
	prefix_mag_7dt = 'USER_7DT_Edmund_mag_'
	prefix_flux_7dt = 'USER_7DT_Edmund_flux_'
	prefix_fluxerr_7dt = 'USER_7DT_Edmund_flux_error_'
	#------------------------------------------------------------
	#	Rename the columns
	#------------------------------------------------------------
	key_to_rename_dict = {}
	#	SDSS
	for filte in filterlist_sdss:
		#	New Values
		# _mag = merged_df[f"{prefix_mag_sdss}{filte}"]
		_flux = merged_df[f"{prefix_flux_sdss}{filte}"]
		_fluxerr = merged_df[f"{prefix_fluxerr_sdss}{filte}"]
		_snr = calculate_snr(_flux, _fluxerr)
		_magerr = calculate_magnitude_error(_flux, _fluxerr)

		#	Former Keywords
		magkey = f"{prefix_mag_sdss}{filte}"
		fluxkey = f"{prefix_flux_sdss}{filte}"
		fluxerrkey = f"{prefix_fluxerr_sdss}{filte}"

		#	New Keywords
		newmagkey = f"{filte}_mag"
		newmagerrkey = f"{filte}_magerr"
		newfluxkey = f"{filte}_flux"
		newfluxerrkey = f"{filte}_fluxerr"
		newsnrkey = f"{filte}_snr"

		#	Keywords to Rename
		key_to_rename_dict[magkey] = newmagkey
		key_to_rename_dict[fluxkey] = newfluxkey
		key_to_rename_dict[fluxerrkey] = newfluxerrkey

		merged_df[newmagerrkey] = _magerr
		merged_df[newsnrkey] = _snr
	#	7DT
	for filte in filterlist_7dt:
		#	New Values
		# _mag = merged_df[f"{prefix_mag_7dt}{filte}"]
		_flux = merged_df[f"{prefix_flux_7dt}{filte}"]
		_fluxerr = merged_df[f"{prefix_fluxerr_7dt}{filte}"]
		_snr = calculate_snr(_flux, _fluxerr)
		_magerr = calculate_magnitude_error(_flux, _fluxerr)

		#	Former Keywords
		magkey = f"{prefix_mag_7dt}{filte}"
		fluxkey = f"{prefix_flux_7dt}{filte}"
		fluxerrkey = f"{prefix_fluxerr_7dt}{filte}"

		#	New Keywords
		if '_50' in filte:
			replace_key = '_50'
			suffix = 'w'
		elif '_25' in filte:
			replace_key = '_25'
			suffix = ''
		else:
			replace_key = ''
			suffix = ''

		newmagkey = f"m{filte.replace(replace_key, suffix)}_mag"
		newmagerrkey = f"m{filte.replace(replace_key, suffix)}_magerr"
		newfluxkey = f"m{filte.replace(replace_key, suffix)}_flux"
		newfluxerrkey = f"m{filte.replace(replace_key, suffix)}_fluxerr"
		newsnrkey = f"m{filte.replace(replace_key, suffix)}_snr"

		#	Keywords to Rename
		key_to_rename_dict[magkey] = newmagkey
		key_to_rename_dict[fluxkey] = newfluxkey
		key_to_rename_dict[fluxerrkey] = newfluxerrkey

		merged_df[newmagerrkey] = _magerr
		merged_df[newsnrkey] = _snr

	#------------------------------------------------------------
	#	Final Dataframe
	#------------------------------------------------------------
	merged_df.rename(columns=key_to_rename_dict, inplace=True)
	return merged_df

In [146]:
fname

'XP_CONTINUOUS_RAW_test.csv'